In [2]:
from dotenv import load_dotenv
from os import getenv
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Set up environment variables
load_dotenv()
data_dir = getenv("DATA_DIR")
azure_key = getenv("AZURE_KEY")
azure_endpoint = getenv("AZURE_ENDPOINT")

In [4]:
# Authenticate the client using your key and endpoint
def azure_auth_client ():
    ta_credential = AzureKeyCredential (azure_key)
    text_analytics_client = TextAnalyticsClient (
        endpoint=azure_endpoint,
        credential=ta_credential)
    return text_analytics_client

In [5]:

azure_client = azure_auth_client()

In [6]:
def azure_health(client, documents):
    poller = client.begin_analyze_healthcare_entities(documents)
    result = poller.result()

    docs = [doc for doc in result if not doc.is_error]

    for idx, doc in enumerate(docs):
        for entity in doc.entities:
            print("Entity: {}".format(entity.text))
            print("...Normalized Text: {}".format(entity.normalized_text))
            print("...Category: {}".format(entity.category))
            print("...Subcategory: {}".format(entity.subcategory))
            print("...Offset: {}".format(entity.offset))
            print("...Confidence score: {}".format(entity.confidence_score))
        for relation in doc.entity_relations:
            print("Relation of type: {} has the following roles".format(relation.relation_type))
            for role in relation.roles:
                print("...Role '{}' with entity '{}'".format(role.name, role.entity.text))
        print("------------------------------------------")

In [7]:
# Test the function
documents = [
    """
    Patient needs to take 50 mg of ibuprofen.
    """
]

azure_health(azure_client, documents)

Entity: 50 mg
...Normalized Text: None
...Category: Dosage
...Subcategory: None
...Offset: 27
...Confidence score: 1.0
Entity: ibuprofen
...Normalized Text: ibuprofen
...Category: MedicationName
...Subcategory: None
...Offset: 36
...Confidence score: 1.0
Relation of type: DosageOfMedication has the following roles
...Role 'Dosage' with entity '50 mg'
...Role 'Medication' with entity 'ibuprofen'
------------------------------------------
